In [1]:
!pip install  pandas==1.4.0 numpy==1.20.0
!pip install keras_self_attention nbdev 
#!pip install torchaudio==0.10.1 pyyaml h5py nvidia_smi keras==2.9.0rc1 tensorflow numba 
!pip install  librosa matplotlib pickle5 Pillow opencv-python nvidia_smi librosa
!pip install tensorflow_addons
#!pip install attention

In [2]:
#export
from tensorflow.tools.docs import doc_controls
import librosa, librosa.display
import matplotlib.pyplot as plt

import os
import glob
import cv2
import pickle5 as pickle
from PIL import Image as im
import argparse
import math
import sys
import time
import copy
import numpy as np
import keras
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization #, regularizers
from keras.layers.noise import GaussianNoise
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D, MaxPooling1D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
import tensorflow as tf
import models_custom 
from keras.models import load_model
import pandas as pd
import models_custom as mcs
from sklearn.model_selection import train_test_split


In [3]:
#default_exp main
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
mirrored_strategy = tf.distribute.MirroredStrategy() #tf.distribute.MultiWorkerMirroredStrategy() #

Num GPUs Available:  1
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [4]:
#export
import pickle5 as pickle
import importlib
importlib.reload(models_custom)



<module 'models_custom' from '/home/ubuntu/pract/lambda/models_custom.py'>

In [5]:

def runme(X_train,val_x0, y_val_wav0, test_x0, y_test_wav0,therapist,emotion,name,folder, conn, use_existing_model='no',  model=None):
    #export
    import warnings
    warnings.filterwarnings('ignore')
    
    pathG='../data'
    if therapist=='Yared Alemu' and emotion in ['fear','sadness']:
        blg=0
    else:
        blg=1
    blg=0
    df = models_custom.create_label(therapist,emotion, pathG,balanced=blg)
    df = df.reset_index()
    df['name2'] =df['name'].apply (lambda x: x.split(".")[0]+".pickle")
    df = df.reset_index() # This redundant did that to avoid some errors




    val_x, y_val = models_custom.moddf(df,val_x0, y_val_wav0 )
    test_x, y_test = models_custom.moddf(df,test_x0, y_test_wav0 )
    val_x0=None 
    y_val_wav0 = None 
    test_x0 =None
    y_test_wav0= None
    filenames= glob.glob(datafolder+"/*.pickle" )
    filenames = [os.path.basename(x) for x in filenames]
    filenames.remove('xvaltest.pickle')

    #export
    


    import importlib
    import time

    # ts stores the time in seconds
    ts = time.time()
    #print("time is :"+ str(ts))

    import sqlite3
    conn = sqlite3.connect('adblog.db')
    therp = therapist.replace(" ","")
    tbl=folder+'_'+therp+'_'+ str(emotion)+'_'+name
    '''
    try:
        sql ="Drop table " + tbl 
        cur = conn.cursor()
        cur.execute(sql)
    except:   
    '''
    importlib.reload(models_custom)
    epochs=100
    model=None
    k=0
    ht={}
    for file in filenames:
        fl = datafolder+'/'+ file    
        with open(fl,"rb") as f:
            train_x0, y_train_wav0 =pickle.load(f)        
        train_x, y_train = models_custom.moddf(df,train_x0,  y_train_wav0) 

        #print(train_x.shape, y_train.shape)

        modname ='../saved_model/'+folder+'_'+therp+'_'+emotion+'_'+name

       
        key = folder+'-'+therp+'-'+ str(emotion)+'-'+name + '-'+str(k)
        key0 = folder+"-"+therp+'-'+ str(emotion)+'-'+name 
        #try:
        #print(fl, key)
        if ( use_existing_model=='yes'):
            print("using the existing model")            
            model = model #load_model(model_path, custom_objects=SeqSelfAttention.get_custom_objects())
        print(train_x.shape, y_train.shape, val_x.shape, y_val.shape)
        try:
            model, history= models_custom.call_model( use_existing_model,model,therp, emotion, train_x, y_train, val_x, y_val, train_x.shape[0],  modname, name, epochs  ) 
        except Exception as e:
            print(str(e))
            break
        #print(history.history.keys())

        #sqldf= pd.DataFrame((history.history['val_loss'], history.history['val_recall'], history.history['val_binary_accuracy'], history.history['val_precision']), index=None).T
        #sqldf=sqldf.reset_index()
        #sqldf.columns=['epoch','val_loss', 'val_recall', 'val_binary_accuracy', 'val_precision']  
        sqldf= pd.DataFrame(history.history)
        sqldf=sqldf.reset_index()
        sqldf=sqldf.rename(columns={"index": "epoch"})
        sqldf['dataset']=folder
        sqldf['therapist']=therp
        sqldf['emotion']=emotion
        sqldf['modelname']=name
        sqldf['valtest'] ='val'
        sqldf['batch']=k
        sqldf['timestamp'] =ts
        sqldf['key0']=key0
        #dict_keys(['loss', 'recall', 'binary_accuracy', 'precision', 'f1_score', 'val_loss', 'val_recall', 'val_binary_accuracy', 'val_precision', 'val_f1_score'])
        sqldf.columns=['epoch', 'loss', 'recall', 'binary_accuracy', 'precision', 'f1_score','val_loss', 'val_recall', 'val_binary_accuracy', 
                     'val_precision', 'val_f1_score', 'dataset', 'therapist', 'emotion', 'modelname', 'valtest', 'batch', 'timestamp', 'key0']
        
        #print(sqldf.shape)
        sqldf.to_sql(tbl,if_exists='append', con=conn, index=None)


        use_existing_model=='yes'


        k=k+1
        #break
    use_existing_model='no'
    return model, df






In [6]:
ths= ["Michelle Lyn", "Sedara Burson"]
ems= ["fear", "anger","sadness"]
names=["LSTM_Attn", "shallow","conv1dlstmAttn", "3CNN2LSTM_Attn"] #,"3CNN2LSTM_Attn" ] #"LSTM_Attn","conv1dlstmAttn" ,"3CNN2LSTM_Attn"

folders =['imgs10','stats',"three_mfcc"] #["LSTM_Attn"]

#folder ='three'

#folder ='imgs10'

#folder = 'stats'


#folder="three_mfcc"
# https://dontrepeatyourself.org/post/overfitting-and-underfitting-in-deep-learning/

In [7]:
import importlib
importlib.reload(models_custom)
import sqlite3
import pandas as pd
conn = sqlite3.connect('adblog.db')

dfk = pd.read_sql("SELECT distinct  dataset, therapist, emotion, modelname from consol ", conn)
dfk["key"] =dfk["dataset"]+ dfk["therapist"]+dfk["emotion"]+dfk["modelname"]
print(list(dfk["key"]))
conn.close()
for folder in folders:
    pathG="../data"
    datafolder=pathG+'/w2v2/'+folder+'/batches'
    file='xvaltest.pickle'

    fl = datafolder+'/'+ file
    with open(fl,"rb") as f:
        X_train,val_x0, y_val_wav0, test_x0, y_test_wav0=pickle.load(f)
        #print(val_x0.shape)
    for therapist in ths:
        for emotion in ems:
            for name in names:
                #if therapist =="Michelle Lyn" and emotion == "anger":
                itm=folder+therapist.replace(" ","")+emotion+name
                print(itm)
                if itm  not in list(dfk["key"]):
                    try:
                        #print(therapist,folder,emotion,name)
                        with mirrored_strategy.scope():
                            model, df = runme(X_train,val_x0, y_val_wav0, test_x0, y_test_wav0,therapist,emotion,name,folder, conn, use_existing_model='no', model=None)
                        test_x, y_test = models_custom.moddf(df,test_x0, y_test_wav0 )
                        score = model.evaluate(test_x, y_test, verbose=1)
                        flr='../data/hist/results.txt'
                        #print(flr)
                        f = open(flr, "a")
                        f.write(folder+" |"+therapist+" |"+emotion+" |"+name+" |"+'\n')
                        f.write(str(score[0])+" |"+ str(score[1]) +" |"+ str(score[2])+" |"+ str(score[3])+ '\n')
                        f.close()
                    except Exception as e:
                        flr='../data/hist/runerror.txt'
                        #print(flr)
                        f = open(flr, "a")
                        print(str(e))
                        f.write(therapist+folder+emotion+name+str(e)+'\n')
                        f.close()

conn.close()


['imgs10MichelleLynfear3CNN2LSTM_Attn', 'imgs10MichelleLynfearconv1dlstmAttn', 'imgs10MichelleLynangerconv1dlstmAttn', 'imgs10MichelleLynanger3CNN2LSTM_Attn', 'imgs10MichelleLynsadnessconv1dlstmAttn', 'imgs10MichelleLynsadness3CNN2LSTM_Attn', 'imgs10SedaraBursonfearconv1dlstmAttn', 'imgs10SedaraBursonfear3CNN2LSTM_Attn', 'imgs10SedaraBursonangerconv1dlstmAttn', 'imgs10SedaraBursonanger3CNN2LSTM_Attn', 'imgs10SedaraBursonsadnessconv1dlstmAttn', 'imgs10SedaraBursonsadness3CNN2LSTM_Attn', 'three_mfccMichelleLynfear3CNN2LSTM_Attn', 'three_mfccMichelleLynanger3CNN2LSTM_Attn', 'three_mfccSedaraBursonanger3CNN2LSTM_Attn', 'three_mfccSedaraBursonsadness3CNN2LSTM_Attn', 'imgs10MichelleLynfearLSTM_Attn', 'imgs10MichelleLynfearshallow', 'imgs10MichelleLynangerLSTM_Attn', 'imgs10MichelleLynangershallow', 'imgs10MichelleLynsadnessLSTM_Attn', 'imgs10MichelleLynsadnessshallow', 'imgs10SedaraBursonfearLSTM_Attn', 'imgs10SedaraBursonfearshallow', 'imgs10SedaraBursonangerLSTM_Attn', 'imgs10SedaraBursona